# Medical Text

Medical Text Dataset [https://www.kaggle.com/datasets/chaitanyakck/medical-text/data]

In [73]:
import pandas as pd
import nltk
from spacy import displacy
import spacy
from transformers import pipeline
import pickle
import json
import os

In [2]:
df = pd.read_csv('data/train.dat', sep="\t", header=None)

In [3]:
df.rename(columns={0:'condition', 1:'abstract'}, inplace=True)
df.head()

,condition,abstract
0,4,Catheterization laboratory events and hospital...
1,5,Renal abscess in children. Three cases of rena...
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...
3,5,Subclavian artery to innominate vein fistula a...
4,4,Effect of local inhibition of gamma-aminobutyr...


In [4]:
lexical_df = df.copy() # a df to compute lexical analysis on

## Lexical Analysis
Lexical analysis consists of the operations of:
- Sentence Splitting
- Tokenization
- Stemming
- Lemmatization
- POS Tagging

It focuses on the main components of a text (words), and aims to recognize them in relation to the context in which they are used, such as sentences or clauses.

### Sentence Splitting
The technique aims to identify the beginning and end of a textual fragment (sentence or clause) with informative content, even if simple.

To achieve this, it uses orthographic features of words (e.g., uppercase initial letters) and delimiters (e.g., punctuation).

In [5]:
# Split the text into sentences
sentences = lexical_df['abstract'].apply(nltk.sent_tokenize)

In [6]:
sentences[0][2] # Print the third sentence of the first record

'Cardiogenic shock was present in eight patients with infarction of the left anterior descending coronary artery, four with infarction of the right coronary artery, and four with infarction of the circumflex coronary artery.'

In [7]:
lexical_df["sentences"] = sentences # set sentences as df col to save progress

### Word Tokenization
The goal of tokenization is to pinpoint the starting and ending positions of each token, whether it’s a word, a number, or a combination of symbols.

As with sentence splitting, the process relies on orthographic features (e.g., initial capital letters) and delimiters (e.g., punctuation).

In [8]:
tokens = []
for record in sentences:
    words = [nltk.word_tokenize(sentence) for sentence in record]
    tokens.append(words)
lexical_df["tokens"] = tokens # set tokenized sentences as df col to save progress

In [9]:
lexical_df.head()

,condition,abstract,sentences,tokens
0,4,Catheterization laboratory events and hospital...,[Catheterization laboratory events and hospita...,"[[Catheterization, laboratory, events, and, ho..."
1,5,Renal abscess in children. Three cases of rena...,"[Renal abscess in children., Three cases of re...","[[Renal, abscess, in, children, .], [Three, ca..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,[Hyperplastic polyps seen at sigmoidoscopy are...,"[[Hyperplastic, polyps, seen, at, sigmoidoscop..."
3,5,Subclavian artery to innominate vein fistula a...,[Subclavian artery to innominate vein fistula ...,"[[Subclavian, artery, to, innominate, vein, fi..."
4,4,Effect of local inhibition of gamma-aminobutyr...,[Effect of local inhibition of gamma-aminobuty...,"[[Effect, of, local, inhibition, of, gamma-ami..."


### Lemmatization
Post-tokenization techniques address the morphological analysis of word-tokens.

Lemmatization identifies the base form (lemma) of inflected words, preserving their meaning and grammatical category. For example, the token _liked_ maps to the lemma *like*.

This process minimizes lexical variation by consolidating different forms of the same word into a unified representation.

In [10]:
wnl = nltk.WordNetLemmatizer()
lemmatization = []
for record in lexical_df['tokens']:
    lemmatized_record = []
    for words in record:
        lemmatized_record.append([wnl.lemmatize(word) for word in words]) # update sentences into lemmatized
    lemmatization.append(lemmatized_record)
lexical_df["lemmatization"] = lemmatization

In [11]:
lexical_df.head()

,condition,abstract,sentences,tokens,lemmatization
0,4,Catheterization laboratory events and hospital...,[Catheterization laboratory events and hospita...,"[[Catheterization, laboratory, events, and, ho...","[[Catheterization, laboratory, event, and, hos..."
1,5,Renal abscess in children. Three cases of rena...,"[Renal abscess in children., Three cases of re...","[[Renal, abscess, in, children, .], [Three, ca...","[[Renal, abscess, in, child, .], [Three, case,..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,[Hyperplastic polyps seen at sigmoidoscopy are...,"[[Hyperplastic, polyps, seen, at, sigmoidoscop...","[[Hyperplastic, polyp, seen, at, sigmoidoscopy..."
3,5,Subclavian artery to innominate vein fistula a...,[Subclavian artery to innominate vein fistula ...,"[[Subclavian, artery, to, innominate, vein, fi...","[[Subclavian, artery, to, innominate, vein, fi..."
4,4,Effect of local inhibition of gamma-aminobutyr...,[Effect of local inhibition of gamma-aminobuty...,"[[Effect, of, local, inhibition, of, gamma-ami...","[[Effect, of, local, inhibition, of, gamma-ami..."


### Stemming
Stemming, like lemmatization, processes inflected forms but reduces them to their root, which may not correspond to a dictionary word.
Unlike lemmatization, it focuses on inflections that create new words and may change the grammatical class, such as *probable* (adjective) stemming to *probably* (adverb).

In [12]:
porterStemmer = nltk.PorterStemmer()
stemming = []
for record in lexical_df['tokens']:
    stemmed_record = [] # stemmed sentences for each record
    for words in record:
        stemmed_record.append([porterStemmer.stem(word) for word in words]) # update sentences into stemmed
    stemming.append(stemmed_record)

lexical_df["stemming"] = stemming

In [13]:
lexical_df.head()

,condition,abstract,sentences,tokens,lemmatization,stemming
0,4,Catheterization laboratory events and hospital...,[Catheterization laboratory events and hospita...,"[[Catheterization, laboratory, events, and, ho...","[[Catheterization, laboratory, event, and, hos...","[[catheter, laboratori, event, and, hospit, ou..."
1,5,Renal abscess in children. Three cases of rena...,"[Renal abscess in children., Three cases of re...","[[Renal, abscess, in, children, .], [Three, ca...","[[Renal, abscess, in, child, .], [Three, case,...","[[renal, abscess, in, children, .], [three, ca..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,[Hyperplastic polyps seen at sigmoidoscopy are...,"[[Hyperplastic, polyps, seen, at, sigmoidoscop...","[[Hyperplastic, polyp, seen, at, sigmoidoscopy...","[[hyperplast, polyp, seen, at, sigmoidoscopi, ..."
3,5,Subclavian artery to innominate vein fistula a...,[Subclavian artery to innominate vein fistula ...,"[[Subclavian, artery, to, innominate, vein, fi...","[[Subclavian, artery, to, innominate, vein, fi...","[[subclavian, arteri, to, innomin, vein, fistu..."
4,4,Effect of local inhibition of gamma-aminobutyr...,[Effect of local inhibition of gamma-aminobuty...,"[[Effect, of, local, inhibition, of, gamma-ami...","[[Effect, of, local, inhibition, of, gamma-ami...","[[effect, of, local, inhibit, of, gamma-aminob..."


### POS tagging
Part-of-speech (POS) tagging assigns a grammatical category to each token, such as noun, verb, or adjective.

In [14]:
pos = []
for sentence in lexical_df['tokens']:
    pos.append([nltk.pos_tag(token) for token in sentence])
lexical_df["pos_tagging"] = pos

In [15]:
lexical_df.head()

,condition,abstract,sentences,tokens,lemmatization,stemming,pos_tagging
0,4,Catheterization laboratory events and hospital...,[Catheterization laboratory events and hospita...,"[[Catheterization, laboratory, events, and, ho...","[[Catheterization, laboratory, event, and, hos...","[[catheter, laboratori, event, and, hospit, ou...","[[(Catheterization, NNP), (laboratory, NN), (e..."
1,5,Renal abscess in children. Three cases of rena...,"[Renal abscess in children., Three cases of re...","[[Renal, abscess, in, children, .], [Three, ca...","[[Renal, abscess, in, child, .], [Three, case,...","[[renal, abscess, in, children, .], [three, ca...","[[(Renal, JJ), (abscess, NN), (in, IN), (child..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,[Hyperplastic polyps seen at sigmoidoscopy are...,"[[Hyperplastic, polyps, seen, at, sigmoidoscop...","[[Hyperplastic, polyp, seen, at, sigmoidoscopy...","[[hyperplast, polyp, seen, at, sigmoidoscopi, ...","[[(Hyperplastic, JJ), (polyps, NNS), (seen, VB..."
3,5,Subclavian artery to innominate vein fistula a...,[Subclavian artery to innominate vein fistula ...,"[[Subclavian, artery, to, innominate, vein, fi...","[[Subclavian, artery, to, innominate, vein, fi...","[[subclavian, arteri, to, innomin, vein, fistu...","[[(Subclavian, JJ), (artery, NN), (to, TO), (i..."
4,4,Effect of local inhibition of gamma-aminobutyr...,[Effect of local inhibition of gamma-aminobuty...,"[[Effect, of, local, inhibition, of, gamma-ami...","[[Effect, of, local, inhibition, of, gamma-ami...","[[effect, of, local, inhibit, of, gamma-aminob...","[[(Effect, NNP), (of, IN), (local, JJ), (inhib..."


### Stop-words Removal
Stop-words are common words that do not carry specific meaning, such as articles, prepositions, and conjunctions.
It is usually performed after lexical analysis to avoid inaccuracies in subsequent syntactic or semantic analyses.

In [16]:
# nltk.download('stopwords')

In [17]:
stopwords = nltk.corpus.stopwords.words('english')

stopwords_removal = []

for record in lexical_df['tokens']:
    filtered_record = []
    for sentence in record:
        filtered_sentence = [word for word in sentence if word.lower() not in stopwords]
        filtered_record.append(filtered_sentence)

    stopwords_removal.append(filtered_record)

lexical_df["stopwords_removal"] = stopwords_removal

In [18]:
lexical_df.head()

,condition,abstract,sentences,tokens,lemmatization,stemming,pos_tagging,stopwords_removal
0,4,Catheterization laboratory events and hospital...,[Catheterization laboratory events and hospita...,"[[Catheterization, laboratory, events, and, ho...","[[Catheterization, laboratory, event, and, hos...","[[catheter, laboratori, event, and, hospit, ou...","[[(Catheterization, NNP), (laboratory, NN), (e...","[[Catheterization, laboratory, events, hospita..."
1,5,Renal abscess in children. Three cases of rena...,"[Renal abscess in children., Three cases of re...","[[Renal, abscess, in, children, .], [Three, ca...","[[Renal, abscess, in, child, .], [Three, case,...","[[renal, abscess, in, children, .], [three, ca...","[[(Renal, JJ), (abscess, NN), (in, IN), (child...","[[Renal, abscess, children, .], [Three, cases,..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,[Hyperplastic polyps seen at sigmoidoscopy are...,"[[Hyperplastic, polyps, seen, at, sigmoidoscop...","[[Hyperplastic, polyp, seen, at, sigmoidoscopy...","[[hyperplast, polyp, seen, at, sigmoidoscopi, ...","[[(Hyperplastic, JJ), (polyps, NNS), (seen, VB...","[[Hyperplastic, polyps, seen, sigmoidoscopy, m..."
3,5,Subclavian artery to innominate vein fistula a...,[Subclavian artery to innominate vein fistula ...,"[[Subclavian, artery, to, innominate, vein, fi...","[[Subclavian, artery, to, innominate, vein, fi...","[[subclavian, arteri, to, innomin, vein, fistu...","[[(Subclavian, JJ), (artery, NN), (to, TO), (i...","[[Subclavian, artery, innominate, vein, fistul..."
4,4,Effect of local inhibition of gamma-aminobutyr...,[Effect of local inhibition of gamma-aminobuty...,"[[Effect, of, local, inhibition, of, gamma-ami...","[[Effect, of, local, inhibition, of, gamma-ami...","[[effect, of, local, inhibit, of, gamma-aminob...","[[(Effect, NNP), (of, IN), (local, JJ), (inhib...","[[Effect, local, inhibition, gamma-aminobutyri..."


# Syntax Analysis
Syntax analysis consists of:
- Shallow Parsing
- Deep Parsing

### Shallow Parsing
Syntactic parsing extends chunking by generating a parse tree. This tree organizes POS-tagging results as leaf nodes and syntactic structures (often chunks) as intermediate nodes, connected hierarchically without representing specific relationships.


In [19]:
syntax_df = df.copy() # a df to compute syntax analysis on

In [20]:
# Define the grammar and the chunk parser
grammar = "NP: {<NNP><NNP>}"
cp = nltk.RegexpParser(grammar) # chunk parser

# Apply chunking to each record
chunking = []
for record in lexical_df['pos_tagging']:
    chunked_record = [cp.parse(sentence) for sentence in record]

    chunking.append(chunked_record)

# Save the chunking results into the dataframe
syntax_df["shallow_parsing"] = chunking

In [21]:
# Display the dataframe
syntax_df.head()

,condition,abstract,shallow_parsing
0,4,Catheterization laboratory events and hospital...,"[[(Catheterization, NNP), (laboratory, NN), (e..."
1,5,Renal abscess in children. Three cases of rena...,"[[(Renal, JJ), (abscess, NN), (in, IN), (child..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[[(Hyperplastic, JJ), (polyps, NNS), (seen, VB..."
3,5,Subclavian artery to innominate vein fistula a...,"[[(Subclavian, JJ), (artery, NN), (to, TO), (i..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[[(Effect, NNP), (of, IN), (local, JJ), (inhib..."


In [22]:
# Display the chunking result for the first record
print(syntax_df['shallow_parsing'][0][4])
#syntax_df['chunking'][0][4]

(S
  There/EX
  was/VBD
  one/CD
  in-laboratory/JJ
  death/NN
  (/(
  shock/JJ
  patient/NN
  with/IN
  infarction/NN
  of/IN
  the/DT
  left/JJ
  anterior/JJ
  descending/VBG
  coronary/JJ
  artery/NN
  )/)
  ./.)


In [23]:
# To draw the parse tree
#syntax_df['chunking'][1][1].draw()

### Deep Parsing
Differently from _Shallow parsing_, _Deep parsing_ aims to infer dependency relationships between nodes.
The result is a dependency graph which relates words that are syntactically linked.

In [24]:
# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

C:\Users\rubin\anaconda3\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [25]:
deep_parsing = []
for sentences in syntax_df["abstract"]:
    sentence_dep = []
    doc = nlp(sentences)
    for token in doc:
        sentence_dep.append((str(token.text), str(token.dep_), str(token.head.text), str([child for child in token.children])))
        # creates a tuple containing the token, dependency nature, head and all dependents of the token
    deep_parsing.append(sentence_dep)

In [26]:
syntax_df["deep_parsing"] = deep_parsing

In [27]:
syntax_df.head()

,condition,abstract,shallow_parsing,deep_parsing
0,4,Catheterization laboratory events and hospital...,"[[(Catheterization, NNP), (laboratory, NN), (e...","[(Catheterization, compound, events, []), (lab..."
1,5,Renal abscess in children. Three cases of rena...,"[[(Renal, JJ), (abscess, NN), (in, IN), (child...","[(Renal, nsubj, abscess, []), (abscess, ROOT, ..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[[(Hyperplastic, JJ), (polyps, NNS), (seen, VB...","[(Hyperplastic, amod, polyps, []), (polyps, ns..."
3,5,Subclavian artery to innominate vein fistula a...,"[[(Subclavian, JJ), (artery, NN), (to, TO), (i...","[(Subclavian, amod, artery, []), (artery, nsub..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[[(Effect, NNP), (of, IN), (local, JJ), (inhib...","[(Effect, ROOT, Effect, [of, :, study, .]), (o..."


In [28]:
# Seleziona la prima riga del dataframe
sentence = syntax_df["abstract"].iloc[0]

# Analizza la frase
doc = nlp(sentence)

# Visualizza il grafico di dipendenze
displacy.render(doc, style="dep", jupyter=True)

## Semantic Analysis

Semantic analysis aims to extract the meaning of a text, focusing on the relationships between entities and the context in which they appear.

### Entity Extraction

Entity extraction identifies named entities in a text, such as persons, organizations, or locations.

In [29]:
semantic_df = df.copy() # a df to compute semantic analysis on

This code extracts named entities from the "abstract" column of a DataFrame called syntax_df using a NLP model.
For each record, it collects the entities and their labels, storing them in a list of tuples.

In [ ]:
pipe = pipeline("token-classification", model="Clinical-AI-Apollo/Medical-NER", aggregation_strategy='simple')

To get a sense of the entities extracted from the text, we can display the unique entities found in the dataset.

In [51]:
# if the entities are already extracted, load them from the pickle file
if os.path.exists('objects/entities.pkl'):
    with open('objects/entities.pkl', 'rb') as file:
        entities = pickle.load(file)
else:
    entities = []
    for record in semantic_df["abstract"]:
        result = pipe(record)

        record_entities = [(entity['word'], entity['entity_group']) for entity in result]

        entities.append(record_entities)

    # Save the entities to a pickle file
    with open('objects/entities.pkl', 'wb') as file:
        pickle.dump(entities, file)

    # Save the entities to a JSON file
    with open('objects/entities.json', 'w') as file:
        json.dump(entities, file)

In [58]:
# Deserialize from the JSON file
import json

with open('objects/entities.json', 'r') as file:
    entities = json.load(file)

In [59]:
semantic_df["entities"] = entities

In [63]:
# Flatten the list of entities and extract only the labels
all_labels = [label for record in semantic_df["entities"] for _, label in record]

# Get the unique labels
unique_labels = set(all_labels)

In [64]:
# Display the unique entities
unique_labels

{'ADMINISTRATION',
 'AGE',
 'AREA',
 'BIOLOGICAL_STRUCTURE',
 'CLINICAL_EVENT',
 'COREFERENCE',
 'DATE',
 'DETAILED_DESCRIPTION',
 'DIAGNOSTIC_PROCEDURE',
 'DISEASE_DISORDER',
 'DISTANCE',
 'DOSAGE',
 'DURATION',
 'FAMILY_HISTORY',
 'HISTORY',
 'LAB_VALUE',
 'MEDICATION',
 'NONBIOLOGICAL_LOCATION',
 'OTHER_ENTITY',
 'PERSONAL_BACKGROUND',
 'SEVERITY',
 'SEX',
 'SIGN_SYMPTOM',
 'THERAPEUTIC_PROCEDURE',
 'TIME',
 'VOLUME'}

In [66]:
semantic_df.head()

,condition,abstract,entities
0,4,Catheterization laboratory events and hospital...,"[[Catheterization laboratory events, DIAGNOSTI..."
1,5,Renal abscess in children. Three cases of rena...,"[[Renal, BIOLOGICAL_STRUCTURE], [abscess, DISE..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[[Hyperplastic polyps, SIGN_SYMPTOM], [sigmoid..."
3,5,Subclavian artery to innominate vein fistula a...,"[[Subclavian artery to innominate vein, BIOLOG..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[[dorsomedial hypothalamus, BIOLOGICAL_STRUCTU..."


In [38]:
def display_entities(pipe, document):
    """
    Mostra le entità riconosciute utilizzando displacy.render().

    Parametri:
        pipe: Pipeline Hugging Face per il riconoscimento delle entità.
        document (str): Testo da processare.

    Ritorna:
        - Visualizzazione delle entità tramite displacy.
        - Lista di entità e relative etichette.
    """
    # Esegui la pipeline sul documento
    result = pipe(document)

    # Crea un modello SpaCy vuoto per gestire il documento
    nlp = spacy.blank("en")
    doc = nlp.make_doc(document)

    # Aggiungi le entità manualmente
    ents = []
    for entity in result:
        start_char = entity['start']
        end_char = entity['end']
        label = entity['entity_group']
        # Crea manualmente uno span
        span = doc.char_span(start_char, end_char, label=label, alignment_mode="expand")
        if span is not None:
            ents.append(span)

    # Assegna le entità al documento
    doc.ents = ents

    # Visualizza con displacy
    displacy.render(doc, style='ent', jupyter=True)

In [39]:
display_entities(pipe, semantic_df["abstract"].iloc[1])

In [40]:
semantic_df.head()

,condition,abstract,entities
0,4,Catheterization laboratory events and hospital...,"[(100, CARDINAL), (100, CARDINAL), (50, CARDIN..."
1,5,Renal abscess in children. Three cases of rena...,"[(Renal, ORG), (Three, CARDINAL), (23, CARDINA..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[(Polyps, ORG), (185, CARDINAL), (99, CARDINAL..."
3,5,Subclavian artery to innominate vein fistula a...,"[(Subclavian, NORP), (Sixteen, CARDINAL), (onl..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[(GABA, ORG), (GABA, ORG), (15, CARDINAL), (2...."


### Relation Extraction

Relation extraction identifies the connections between entities in a text, such as the subject, object, and verb of a sentence.

This process is more complex than entity extraction, as it requires understanding the syntactic structure of the text to infer relationships between entities.

1. **Iterate through sentences**:
   Process each sentence (`doc`) and its entities, storing entities in a set for faster lookup (`entities_set`).

2. **Extract relations from tokens**:
   Loop through tokens with dependencies like "ROOT" or "VERB" to identify:
   - **Subjects**: Found in children with dependencies like "nsubj" or "agent."
   - **Objects**: Found in children with dependencies like "dobj" or "pobj."

3. **Record relations**:
   - Direct relations: `(subject, verb, object).`
   - Prepositional relations: Handle `prep` and `pobj` to form `(subj, verb_prep, obj)` or similar.

4. **Handle conjunctions**:
   Add relations involving conjunctive tokens (`conj`).



In [67]:
if os.path.exists('objects/relations.pkl'):
    all_relations = pickle.load(open('objects/relations.pkl', 'rb'))
else:
    # Relations Extraction
    all_relations = []

    for sentences, sentence_entities in zip(semantic_df["abstract"], semantic_df["entities"]):
        doc = nlp(sentences)
        sentence_relations = []
        entities_set = {ent_text for ent_text, _ in sentence_entities}  # Use a set for faster lookup

        # Extract relations for each root or verb token
        for token in doc:
            if token.dep_ in ("ROOT", "VERB", "aux", "auxpass"):
                # Find the subject
                subj = None
                for child in token.children:
                    if child.dep_ in ("nsubj", "nsubjpass", "csubj", "agent"):
                        for ent_text in entities_set:
                            if ent_text in child.text or child.text in ent_text:
                                subj = ent_text
                                break

                # Find the object
                obj = None
                for child in token.children:
                    if child.dep_ in ("dobj", "pobj", "iobj", "attr", "acomp"):
                        for ent_text in entities_set:
                            if ent_text in child.text or child.text in ent_text:
                                obj = ent_text
                                break

                # Record direct relations
                if subj and obj:
                    sentence_relations.append((subj, token.lemma_, obj))

                # Handle prepositional relations
                for child in token.children:
                    if child.dep_ == "prep":
                        for pobj in child.children:
                            if pobj.dep_ == "pobj":
                                for ent_text in entities_set:
                                    if ent_text in pobj.text or pobj.text in ent_text:
                                        if subj:
                                            sentence_relations.append((subj, f"{token.lemma_}_{child.text}", ent_text))
                                        elif obj:  # Use the object if no subject
                                            sentence_relations.append((obj, f"{token.lemma_}_{child.text}", ent_text))

                # Handle conjunctive subjects and objects
                for child in token.children:
                    if child.dep_ == "conj" and subj:
                        sentence_relations.append((subj, token.lemma_, child.text))

        # Post-process to remove duplicates
        sentence_relations = list(set(sentence_relations))

        # Add relations to the main list
        all_relations.append(sentence_relations)

    # Save the relations to a pickle file
    with open('objects/relations.pkl', 'wb') as file:
        pickle.dump(all_relations, file)

    # Save the relations to a JSON file
    with open('objects/relations.json', 'w') as file:
        json.dump(all_relations, file)

In [68]:
all_relations

[[('Cardiogenic shock', 'be_in', 's'),
  ('s', 'be', 's'),
  ('s', 'assess_in', 's'),
  ('s', 'occur_in', 's'),
  ('s', 'occur_with', 'myocardial infarction'),
  ('s', 'assess_with', 's'),
  ('s', 'assess_with', 'infarct'),
  ('s', 'assess', 'arteries'),
  ('s', 'assess_with', 'infarction'),
  ('s', 'occur_with', 'infarct'),
  ('s', 'occur_with', 'infarction'),
  ('Cardiogenic shock', 'be', 's')],
 [('Fever', 'be', 'associated'),
  ('Antibiotics', 'produce', 'required'),
  ('Reflux', 'document', 'had')],
 [('Hyperplastic polyps', 'remove_at', 'sigmoidoscopy'),
  ('Hyperplastic polyps', 'remove_at', 'sigmoid'),
  ('Hyperplastic polyps', 'remove_at', 'oid'),
  ('adenoma', 'have_at', 'proximal sites')],
 [('Subclavian artery to innominate vein', 'innominate', 'fistula'),
  ('Subclavian artery to innominate vein', 'innominate_after', 's')],
 [('microdialysis', 'cause', 'variable increases'),
  ('perfusion', 'cause', 'variable increases'),
  ('microdialysis', 'cause', 'change'),
  ('variabl

In [69]:
# Add the all_relations list as a column to semantic_df
semantic_df['relations'] = [relations if relations else pd.NA for relations in all_relations]

In [70]:
# Display the updated dataframe
semantic_df[semantic_df['relations'].notna()]

,condition,abstract,entities,relations
0,4,Catheterization laboratory events and hospital...,"[[Catheterization laboratory events, DIAGNOSTI...","[(Cardiogenic shock, be_in, s), (s, be, s), (s..."
1,5,Renal abscess in children. Three cases of rena...,"[[Renal, BIOLOGICAL_STRUCTURE], [abscess, DISE...","[(Fever, be, associated), (Antibiotics, produc..."
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...,"[[Hyperplastic polyps, SIGN_SYMPTOM], [sigmoid...","[(Hyperplastic polyps, remove_at, sigmoidoscop..."
3,5,Subclavian artery to innominate vein fistula a...,"[[Subclavian artery to innominate vein, BIOLOG...","[(Subclavian artery to innominate vein, innomi..."
4,4,Effect of local inhibition of gamma-aminobutyr...,"[[dorsomedial hypothalamus, BIOLOGICAL_STRUCTU...","[(microdialysis, cause, variable increases), (..."
...,...,...,...,...
14430,5,Extensive peripheral retinectomy combined with...,"[[Extensive, DETAILED_DESCRIPTION], [periphera...","[(normotensive, be_Of, eyes), (retinal detachm..."
14432,4,Embolization with particles in thoracic intram...,"[[Embolization, THERAPEUTIC_PROCEDURE], [parti...","[(up, be, 1-15 years), (Clinical aggravation, ..."
14434,1,Mammographic measurements before and after aug...,"[[Mammographic measurements, DIAGNOSTIC_PROCED...","[(Anterior breast tissue, see_with, compressio..."
14436,2,Complications of Tenckhoff catheters post remo...,"[[ckhoff, DISEASE_DISORDER], [January 1979 to,...","[(D, include, monitored), (peritoneal dialysis..."
